In [52]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, roc_curve,auc

In [53]:
# read Excel data and combine into one data structure
data = pd.read_excel("Handbag Data_KP5.xlsx")

In [112]:
# Make sure 'Month' column exists
data['Paid at'] = pd.to_datetime(data['Paid at'], errors='coerce')  # Handle errors just in case
data['Month'] = data['Paid at'].dt.month

# Convert 'Month' to string to make sure we get separate columns for each month
data['Month'] = data['Month'].astype(str)

# Generate dummy variables for each month
month_dummies = pd.get_dummies(data['Month'], prefix='Month')

# Concatenate the dummy variables with the original DataFrame
data = pd.concat([data, month_dummies], axis=1)

# Now 'data' contains a set of new columns: 'Month_1', 'Month_2', ..., 'Month_12',
# each indicating the presence of a month with 0 or 1.

# Optionally, you can check the result
print(data.head())

      Name Financial Status  Expired  Paid  Partially_Paid  \
0  #136493             paid        0     1               0   
1  #136491             paid        0     1               0   
2  #136490         refunded        0     0               0   
3  #136487             paid        0     1               0   
4  #136486             paid        0     1               0   

   Partially_Refunded  Pending  Refunded  Voided  Successful_Order  ...  \
0                   0        0         0       0                 1  ...   
1                   0        0         0       0                 1  ...   
2                   0        0         1       0                 0  ...   
3                   0        0         0       0                 1  ...   
4                   0        0         0       0                 1  ...   

  Month_11  Month_12  Month_2  Month_3  Month_4  Month_5  Month_6  Month_7  \
0        0         0        1        0        0        0        0        0   
1        1         0

In [113]:
# Define the state to region mapping
state_to_region = {
    'AL': 'South', 'AK': 'West', 'AZ': 'West', 'AR': 'South', 'CA': 'West',
    'CO': 'West', 'CT': 'East', 'DE': 'East', 'FL': 'South', 'GA': 'South',
    'HI': 'West', 'ID': 'West', 'IL': 'Midwest', 'IN': 'Midwest', 'IA': 'Midwest',
    'KS': 'Midwest', 'KY': 'South', 'LA': 'South', 'ME': 'East', 'MD': 'East',
    'MA': 'East', 'MI': 'Midwest', 'MN': 'Midwest', 'MS': 'South', 'MO': 'Midwest',
    'MT': 'West', 'NE': 'Midwest', 'NV': 'West', 'NH': 'East', 'NJ': 'East',
    'NM': 'West', 'NY': 'East', 'NC': 'South', 'ND': 'Midwest', 'OH': 'Midwest',
    'OK': 'South', 'OR': 'West', 'PA': 'East', 'RI': 'East', 'SC': 'South',
    'SD': 'Midwest', 'TN': 'South', 'TX': 'South', 'UT': 'West', 'VT': 'East',
    'VA': 'South', 'WA': 'West', 'WV': 'South', 'WI': 'Midwest', 'WY': 'West',
    'DC': 'East', 'PR': 'South'
}

In [114]:
df = pd.DataFrame(data)

# Function to map state to region
def map_state_to_region(state):
    return state_to_region.get(state, 'International')

# Creating a new 'Region' column by applying the mapping function
df['Region'] = df['Shipping Province'].apply(map_state_to_region)

# Function to add dummy variable columns for each region directly to the DataFrame
def add_region_columns(df):
    # Create dummy variables for the regions based on the 'Region' column
    region_dummies = pd.get_dummies(df['Region'])

    # Concatenate the dummy variables with the original DataFrame
    return pd.concat([df, region_dummies], axis=1)

print(df)

          Name    Financial Status  Expired  Paid  Partially_Paid  \
0      #136493                paid        0     1               0   
1      #136491                paid        0     1               0   
2      #136490            refunded        0     0               0   
3      #136487                paid        0     1               0   
4      #136486                paid        0     1               0   
...        ...                 ...      ...   ...             ...   
36007   #97828                paid        0     1               0   
36008   #97827                paid        0     1               0   
36009   #97826  partially_refunded        0     0               0   
36010   #97825  partially_refunded        0     0               0   
36011   #97824                paid        0     1               0   

       Partially_Refunded  Pending  Refunded  Voided  Successful_Order  ...  \
0                       0        0         0       0                 1  ...   
1            

In [115]:
trainIndices = np.arange(21607)
valIndices = np.arange(21608,len(data))
training = df.iloc[trainIndices,:]
validation = df.iloc[valIndices,:]

In [116]:
#formula = 'Successful_Order ~ Bag_Purchased + Last_Chance + Discount + Region + Lineitem_quantity + Subtotal'
formula = 'Successful_Order ~ Number_of_Bags + Last_Chance + Discount + Region + Lineitem_quantity + Subtotal + Month'

In [117]:
# Now, fit the model with the corrected formula
model_success = smf.logit(formula=formula, data=training).fit()

# Predict and print the summary
validationProbs = model_success.predict(validation)
print(model_success.summary())

Optimization terminated successfully.
         Current function value: 0.339155
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       Successful_Order   No. Observations:                21607
Model:                          Logit   Df Residuals:                    21586
Method:                           MLE   Df Model:                           20
Date:                Mon, 11 Mar 2024   Pseudo R-squ.:                 0.07572
Time:                        22:43:05   Log-Likelihood:                -7328.1
converged:                       True   LL-Null:                       -7928.5
Covariance Type:            nonrobust   LLR p-value:                5.264e-242
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.1849      0.152     21.002      0.000       2.888   

In [118]:
auc = roc_auc_score(validation['Successful_Order'], validationProbs)
print('AUC of the logistic regression on the validation set is',auc)

AUC of the logistic regression on the validation set is 0.6681241004764391
